In [ ]:
import toml
import sys
import os
import pandas as pd

 # Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the parent directory to the Python path
sys.path.append(parent_dir)
# from mimetypes import guess_type
from langchain_openai import AzureChatOpenAI
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
 
from chains.image_extractor import ImageExtractChain
# from chains.doc_summarize import DocumentSummarize


In [ ]:
azure_config = toml.load(r"")["configs"][0]
llm = AzureChatOpenAI(
    openai_api_version=azure_config["api_version"],
    azure_deployment=azure_config["deployment_name"],
    api_key=azure_config["api_key"],
    azure_endpoint=azure_config["api_base"],
    model=azure_config["model_name"],
    model_name=azure_config["model_name"],
    temperature=0,
)
response = llm.invoke("hi")

In [ ]:
# Directory containing image files
image_directory = "images"

# List all image files in the directory
image_paths = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Create an instance of ImageExtractChain for each image and extract information
results = {}
for image_path in image_paths:
    # Extract country name from the image file name
    country_name, global_id = os.path.basename(image_path).split('_')[:2]
    countryname_globalid = f"{country_name}_{global_id}"

    # Create an instance of ImageExtractChain
    iec = ImageExtractChain(model=llm, image_path=image_path)
    
    # Extract information from the image
    result = iec.extract_from_image()
    
    # Store the result in the dictionary
    results[countryname_globalid] = result


In [ ]:

# Convert the results dictionary to a DataFrame
df = pd.DataFrame.from_dict(results, orient='index')

# Display the DataFrame
df

In [ ]:
average_measurements = df['point_measurements_description']
for avg in average_measurements:
    print(avg)
    print()